In [59]:
import matplotlib.pyplot as plt
import torch.optim as optim
from Lec7sbs import StepByStep
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import torch
from torchvision.transforms.functional import  rotate, vflip, hflip
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, RandomVerticalFlip, RandomRotation, ToPILImage, Normalize
from torchvision import transforms
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn as nn
Images = np.load('./X.npy')
labels = np.load('./y.npy')
X=  torch.as_tensor(Images/255.0).float()
y=  torch.as_tensor(labels).long()

In [60]:
from torch.utils.data import Dataset
class TransformedTensorDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform
    
    def __getitem__(self, index):
        x = self.x[index]
 
        if self.transform:
            x = self.transform(x)
 
        return x, self.y[index]

    def __len__(self):
        return len(self.x)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle = True)
transform = Normalize(mean=(.5,), std=(.5,))
train_dataset = TransformedTensorDataset(X_train, y_train, transform=transform)
val_dataset = TransformedTensorDataset(X_test, y_test, transform=transform)

In [62]:
X_train.shape

torch.Size([240, 1, 20, 20])

In [63]:
# Generate loaders
def CNNModel(num_layers: int, num_filters: [], filter_size: [], padding: []):
    assert len(num_filters) == num_layers
    assert len(filter_size) == num_layers
    assert len(padding) == num_layers
    x = (20 -filter_size[0] + 2 * padding[0]) + 1
    y = (20 -filter_size[0] + 2 * padding[0]) + 1
    x = x // 2  # Assuming max pool with kernel size 2
    y = y // 2
    cnn_model = nn.Sequential()
    cnn_model.add_module('conv_1', nn.Conv2d(in_channels=1,
                                             out_channels=num_filters[0],
                                             kernel_size=filter_size[0],
                                             padding=padding[0]))
    cnn_model.add_module('relu_1', nn.ReLU())
    cnn_model.add_module('maxpool_1', nn.MaxPool2d(kernel_size=2))
    for i in range(1, num_layers):
        cnn_model.add_module('conv_'+str(i+1), nn.Conv2d(in_channels=num_filters[i-1], out_channels=num_filters[i], kernel_size=filter_size[i], padding=padding[i]))
        cnn_model.add_module('relu_'+str(i+1), nn.ReLU())
        cnn_model.add_module('pool_' + str(i+1), nn.MaxPool2d(kernel_size=2))
        cnn_model.add_module('dropout' + str(i+1), nn.Dropout(p=0.5))
        x = (x - filter_size[i] + 2 * padding[i]) + 1  # Assuming stride=1
        y = (y - filter_size[i] + 2 * padding[i]) + 1
        x = x // 2  # Assuming max pool with kernel size 2
        y = y // 2
    cnn_model.add_module('flat', nn.Flatten())
    cnn_model.add_module('lin', nn.Linear(x*y*num_filters[-1], 3))
    return cnn_model

In [68]:
a = CNNModel(2,[16,32], [2,2], [0,0])

In [70]:
a(X_test)

RuntimeError: Given groups=1, weight of size [32, 16, 2, 2], expected input[60, 1, 20, 20] to have 16 channels, but got 1 channels instead

In [67]:
num_layers = [1,2,3,4]
num_filters = [[16],[32],[64], [16, 32], [16, 64], [32, 64], [32, 128], [16, 32, 64], [16,32,64, 16], [16, 32, 64, 128]]
filter_size = [[2], [3], [5], [2,2], [3, 3],[5,5], [3,3,3], [5,5,5], [3,5,3], [5, 5, 3], [3, 3, 3, 3], [5,5,3,3], [5,5,5,5]]
padding = [[0], [1], [0, 0],[1,1], [1, 1, 1], [0,0,0], [1, 1, 1, 1], [0,0,0,0]]
cnn_models = []
for i in range(len(num_layers)):
    for j in range(len(num_filters)):
        if len(num_filters[j]) != num_layers[i]:
            continue
        for k in range(len(filter_size)):
            if len(filter_size[k]) != num_layers[i]:
                continue
            for l in range(len(padding)):
                if len(padding[l]) != num_layers[i]:
                    continue
                cnn_models.append(CNNModel(num_layers[i], num_filters[j], filter_size[k], padding[l]))
                cnn_models[-1].name = 'CNNModel_' + str(num_layers[i])+ "_" + str(num_filters[j]) +"_"+ str(filter_size[k])+"_" + str(l)

/Users/mariavictoriadacostarivas/opt/anaconda3/envs/TFG/lib/python3.8/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [65]:
from datetime import datetime
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(val_dataset, batch_size=10, shuffle=True)
lr = 1e-2
# Defines a BCE without logits loss function (uses probabilities)
n_epochs = 100
"""
You should have an automated Python program that trains each model for around 20
epochs. You can suppress plotting the losses to avoid cluttering your report. Instead, all
we care about is the following summary out of each model:
o minBC: minimum validation BC loss at the end of training. Totally ignore the
training loss. The validation seems to be more stable.
o E95: number of epochs it took to reach 95% accuracy. If your model has not
reached it, make this number None.
o E90: number… to reach 90%
o E80: number… to reach 80%
o E70: number… to reach 70%
o P: number of parameters in the model
o T: evaluation time for one input. You can determine that by evaluating 100
consecutive entries and dividing it by 100, etc. to eliminate measurement noise.
o Cost: The cost of the model. This is a figure-of-merit that determines how
expensive it is to use the model. Calculate it by 𝐶𝑜𝑠𝑡 = 𝑃 ∗ 𝑇. Since increasing 𝑃
will very likely decrease 𝑇, it is not necessarily obvious which model will have
the lowest Cost. It is common practice to use such metrics; for example, in chip
design, Power*Delay or Area*Delay products are commonly used to study how
the power consumption can be traded-off against delay (i.e., chip speed), etc.
"""
reports = []
for model in cnn_models:
    print(model.name)
    report = {"Config":model.name ,
              "E95": None, "E90": None, "E80": None, "E70": None, "P": None, "T": None, "Cost": None, "minBC": None}
    min_val_loss = float('inf')
    optimizer = optim.SGD(model.parameters(), lr=lr)
    report["P"] = sum(p.numel() for p in model.parameters())
    loss_fn =nn.CrossEntropyLoss(reduction='mean')
    for epoch in range(n_epochs):
        train_loss = []
        val_loss = []
        for data,target in train_loader:
            model.train()
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
        with torch.no_grad():
            for data,target in test_loader:
                model.eval()
                output = model(data)
                loss_val = loss_fn(output, target)
                val_loss.append(loss_val.item())
            output = model(X_test)
            predicted_classes = torch.argmax(output, dim=1)
            accuracy = (predicted_classes == y_test).sum().item()
            accuracy /= len(target)
            if accuracy >= 0.7 and report["E70"] is None:
                report["E70"] = epoch
            if accuracy >= 0.8 and report["E80"] is None:
                report["E80"] = epoch
            if accuracy >= 0.9 and report["E90"] is None:
                report["E90"] = epoch
            if accuracy >= 0.95 and report["E95"] is None:
                report["E95"] = epoch
        train_loss_val = np.mean(train_loss)
        val_loss_val = np.mean(val_loss)
        if val_loss_val < min_val_loss:
            min_val_loss = val_loss_val
        #print(f" Epoch {epoch}: train loss {train_loss_val}, val loss {val_loss_val}")
    start_t = datetime.now()
    output = model(X_test)
    report["T"] = (datetime.now() - start_t).total_seconds() / X_test.shape[0]
    report["minBC"] = min_val_loss
    report["Cost"] = report["P"] * report["T"]
    print(report)
    reports.append(report)





CNNModel_1_[16]_[2]_0
{'Config': 'CNNModel_1_[16]_[2]_0', 'E95': 0, 'E90': 0, 'E80': 0, 'E70': 0, 'P': 3971, 'T': 1.2916666666666666e-05, 'Cost': 0.05129208333333333, 'minBC': 0.16518747713416815}
CNNModel_1_[16]_[2]_1
{'Config': 'CNNModel_1_[16]_[2]_1', 'E95': 0, 'E90': 0, 'E80': 0, 'E70': 0, 'P': 4883, 'T': 2.1133333333333333e-05, 'Cost': 0.10319406666666667, 'minBC': 0.15783592375616232}
CNNModel_1_[16]_[3]_0
{'Config': 'CNNModel_1_[16]_[3]_0', 'E95': 0, 'E90': 0, 'E80': 0, 'E70': 0, 'P': 4051, 'T': 1.615e-05, 'Cost': 0.06542365, 'minBC': 0.14906906600420675}
CNNModel_1_[16]_[3]_1
{'Config': 'CNNModel_1_[16]_[3]_1', 'E95': 0, 'E90': 0, 'E80': 0, 'E70': 0, 'P': 4963, 'T': 2.0666666666666666e-05, 'Cost': 0.10256866666666667, 'minBC': 0.1622285097837448}
CNNModel_1_[16]_[5]_0
{'Config': 'CNNModel_1_[16]_[5]_0', 'E95': 0, 'E90': 0, 'E80': 0, 'E70': 0, 'P': 3491, 'T': 2.5133333333333332e-05, 'Cost': 0.08774046666666667, 'minBC': 0.1548525644466281}
CNNModel_1_[16]_[5]_1
{'Config': 'CNNMo

RuntimeError: Given groups=1, weight of size [32, 16, 2, 2], expected input[10, 1, 20, 20] to have 16 channels, but got 1 channels instead